# Stanford Question Answering Dataset（SQuAD） —— text span答案类型开创者
https://zhuanlan.zhihu.com/p/137828922

SQuAD有两个版本，1.1版本在2016年EMNLP的论文中介绍，2.0版本在2018年ACL的论文中介绍。

论文链接：

SQuAD 1.1: SQuAD: 100,000+ Questions for Machine Comprehension of Text

SQuAD 2.0: Know What You Don’t Know: Unanswerable Questions for SQuAD

数据集下载地址和leaderboard：[The Stanford Question Answering Dataset](https://rajpurkar.github.io/SQuAD-explorer/)



## SQuAD1.0



除了满足大规模，问题更高质量这两个特点。SQuAD也有其他方面的创新。

那就是，答案的形式的创新。

SQuAD的答案是text span，就是以文章原文中的某小一段文字来作为问题的答案。而以往的数据集要么是完形填空题（可以想象成填词题，例如CNN/Daily Mail），要么是多项选择题（例如一个比较经典的叫MCTest的），还有就是没标准答案free form的。

明显的，text span在某些方面，会优于以上三种。

相对于free form来讲， text span有明确的固定答案，更容易衡量model的表现。

多项选择类的问题可能会存在作弊的行为，譬如四选题，怎么着model都能有25%的正确率，再加上答案构成的bias，什么三长一短选最短，三短一长选最长。model鬼精鬼精的，哪晓得它预测出来的那些答案是怎么搞出来的，反正可解释性咱们也不好说。

### 数据集生成

从经典三元素文章-问题-答案（passage-question-answer）开始讲

1. 文章 Passage：文章挑选自英文维基中比较高质量的10000篇维基百科中。先随机抽取了536篇文章，把它们按自然段划分。然后去除图片，表格，和过短的段落。最后保留了23，125个段落来作为数据集的passage。
1. 问题Question：问题由外包工作者人工提出。对于每一个passage，工作者需要提出5个问题，并且标记出文中的text span作为答案。而且还有一个额外的要求，工作者们需要以自己的词汇来提问，而不是简单地复制文章中出现的词，这样可以避免model 直接进行token match的问题。
1. 答案Answer：工作者们标记出来的text span就是问题的答案。
1. 这里额外增加了一个采集步骤，用来衡量人类在这个数据集上的performance。数据集被划分成training set（80%），development set（10%），test set（10%）。在dev和test set上，每个提出来的问题会额外采集另外两个工作人员对于这个问题的答案（text span）。

### 数据集分析

答案的多样性
下图中我们可以看到，答案类型那是很多呀，包含了日期，数值，人物，地点，名词短语，甚至形容词、动词等等。一对比，CNN/Daily Mail简直boring~

回答答案所需的推理能力（reasoning）

为了研究问题所需要的推理能力，作者抽取出小部分问题来人工分析。

所需的推理能力包含了，对词汇变化、语法变化的理解，以及少部分的多句综合推理等。

### 模型表现的衡量方式

这篇论文给出了一个新的答案方式——text span，对应地，它也给出了用来衡量model表现的新方式。

* Exact Match (EM): 就是在所有的问题中，model预测出来的text span与人工给定答案一毛一样的问题的比例。
* F1 score：先计算每个预测答案span与正确答案span之间的重叠比例，然后取所有答案的均值。
* Human Performance：human performance的衡量是将一个答案作为标准答案，另外收集到的答案作为预测答案，然后来计算EM和F1. 因为text span手一抖就多一个或者少一个词了，所以人类的EM和F1也并不是100%。EM是77%，F1是86.8%。这个数据，已然被各种model刷爆。

那么同样地，论文里提到的用来跑这个数据集的model我们这里就不介绍了。

SQuAD1.1 介绍完毕(ง •_•)ง

## SQuAD2.0

以text span为答案的QA数据集，存在一定的缺陷。虽然model可以定位到答案的位置并且抽取出正确答案，但是当一个问题的答案并不存在于给定文章中时，model也会强答，从文本中乱猜一节作为答案，这显然不靠谱。

那么为了解决这个问题，作者设计出了SQuAD的2.0版本~

### 与SQuAD1.1版本的区别

在原来SQuAD1.1的版本上，增加了不可回答的问题（unanswerable questions）。当然这些问题并不是随机生成，或者人工胡乱问出的那种瞎xx扯问题。

这些unanswerable question的生成遵循以下两个原则：

* 相关性Relevance：这些问题尽管不可回答，但需要是与对应的passage的主题密切相关的。要是风马牛不相接的话，model太容易识别了。
* passage中存在看似合理的答案：譬如说，问“哪一间公司成立于1992年？”那么需要对应的passage里存在一些company，尽管不是正确答案，但看似合理，有一定的误导性。这个设计是为了避免model能够直接进行答案类型判断，要是文本里没这个类型的词或者短语，就推测出问题是不是answerable的。都说了model鬼精鬼精的 ┑(￣Д ￣)┍

这里多叨叨一句，后来的某些数据集，其实存在好多可作弊的问题。就是某些问题，通过推测答案类型就能回答出来了。譬如问XXX位于哪个城市，passage里就那么一个城市，咋看答案都是它。那还需要model学到啥理解能力，明显不靠谱。那些model精得很，理解能力不太行，乱猜作弊第一名。

### 数据集生成

与SQuAD1.1一样，是由人工生成。每个工人会看到一篇维基文章，他们针对每个段落需要提出不同的unanswerable问题，并且需要标出文章里看似合理的答案。同时这些工人也能看到1.1版本里，这篇维基文章对应的哪些问题，这样的话有助于工人提出哪些与可回答的问题比较接近的unanswerable问题。

这种标注类的工作时薪大概10美刀左右，就是七十多人民币。在校的大学生小朋友可以试一试~

### 数据集分析

1.Passage-Question-Answer 三兄弟

SQuAD2.0的passage，question的格式与1.1一毛一样。唯一不同的就是多了unanswerable的问题。

那么也就是answer变成了两种类型，一种是直接预测出unanswerable，另一种还是text span（要是可回答的话）

下面是一个实例：对于介绍哈佛的这篇文章，下面的问题有些答案是 text span，有些答案是<No Answer>
    
### Scale

这里需要看下的就是2.0比1.1规模还大了不少。下面这个negative examples就是unanswerable的问题。在training set里，可回答的问题大概是不可回答问题的数量的两倍，在dev和text set里，大概可回答与不可回答比例在1:1左右。这个还是蛮有挑战性的。
![](img/sq01.png)

与1.1一致，同样采用EM和F1的指标

人类表现 Human performance （EM: 86.8; F1: 89.5）

除了原本生成数据集的工人，还额外聘请了一部分工人，对于数据集里的问题进行回答。这些工人可以看到文章和问题，然后根据文章和问题先判断，依据文章内容，问题是否可以回答。如果可回答，标记处text span，如果不可回答，则直接标记unanswerable。

为了降低不确定性带来的偏差，对于每个问题，额外收集了多个答案，平均值为4.8个。这里也是之于1.1版本的改善。

## dev-2.0样例

In [ ]:
{
	"version": "v2.0",
	"data": [
		{
		"title": "Normans",
		"paragraphs": [
			{
			"qas": [
			   {
					"question": "In what country is Normandy located?",
					"id": "56ddde6b9a695914005b9628",
					"answers": [{
						"text": "France",
						"answer_start": 159
					}, {
						"text": "France",
						"answer_start": 159
					}, {
						"text": "France",
						"answer_start": 159
					}, {
						"text": "France",
						"answer_start": 159
					}],
					"is_impossible": false
				},{
					"plausible_answers": [{
						"text": "10th century",
						"answer_start": 671
					}],
					"question": "When did the Frankish identity emerge?",
					"id": "5ad39d53604f3c001a3fe8d4",
					"answers": [],
					"is_impossible": true
				}
			],
			"context": "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."
			}, 
		]
	}]
}

# doccano标注数据转squad数据脚本

In [ ]:
import json

with open('data_origin/project_4_labels.json') as f:
    project_4_labels = json.load(f)

label_dict = {i['id']:i['text'] for i in project_4_labels}   
label_dict

In [ ]:
with open('data_origin/file.json') as f:
    lines = f.readlines()
    
d = dict()

d['version'] = 'v2.0'
d['data'] = []
for i, line in enumerate(lines):
    tmp_ = json.loads(line)
#     print(tmp_['id'])
    sub = dict()
    sub['title'] = f'medical_record{i}'
    sub['paragraphs'] = []
    sub_ = dict()
    sub_['qas'] = []
    sub_['context'] = tmp_['text']
    if len(tmp_['annotations'])==0: continue
    for ii, anno in enumerate(tmp_['annotations']):
        qas = {}
        qas['question'] = label_dict[anno['label']]
        qas['id'] = f'{i}_{ii}'
        qas['answers'] = []
        answer = {}
        
        answer['text'] = sub_['context'][anno['start_offset']:anno['end_offset']]
        answer['answer_start'] = anno['start_offset']
        qas['answers'].append(answer)
        qas['is_impossible'] = False
        sub_['qas'].append(qas) 
    sub['paragraphs'].append(sub_)    
#     print(sub)
    d['data'].append(sub)
#     break

with open('data_gen/med_examples.json', 'w', encoding='utf8') as f:
    json.dump(d, f, ensure_ascii=False)

In [43]:
# 切分context版本+padding
d = dict()

d['version'] = 'v2.0'
d['data'] = []

text_len = 480

for i, line in enumerate(lines):
    tmp_ = json.loads(line)
    if len(tmp_['annotations'])==0: continue
#     print(tmp_['id'])
    sub = dict()
    sub['title'] = f'medical_record{i}'
    sub['paragraphs'] = []
    context_ = tmp_['text']
    
    for iii in range(min(3, math.ceil(len(context_)/text_len))):
        start = iii*text_len
        end = (start+text_len)
        sub_ = dict()
        sub_['qas'] = []
        sub_['context'] = ' '.join(list(context_[start: end]))
        
        for ii, anno in enumerate(tmp_['annotations']):
            if (anno['end_offset'] < start) or (anno['start_offset'] > end): continue
            if anno['end_offset'] < end:
                end_ans = anno['end_offset']
            else:
                end_ans = end
            if anno['start_offset'] > start:
                start_ans = anno['start_offset']
            else:
                start_ans = start
            
            
            qas = {}
            qas['question'] = ' '.join(list(label_dict[anno['label']]))
            qas['id'] = f'{i}_{iii}_{ii}'
            qas['answers'] = []
            answer = {}

            answer['text'] = ' '.join(list(context_[start_ans: end_ans]))
            answer['answer_start'] = (start_ans-start)*2
            qas['answers'].append(answer)
            qas['is_impossible'] = False
            sub_['qas'].append(qas) 
        if sub_['qas']: sub['paragraphs'].append(sub_)    
#     print(sub)
    d['data'].append(sub)
#     break

In [44]:
d

{'version': 'v2.0',
 'data': [{'title': 'medical_record0',
   'paragraphs': [{'qas': [{'question': '现 病 史 / 入 院 情 况',
       'id': '0_0_0',
       'answers': [{'text': '入 院 时 , 患 者 咳 嗽 阵 作 , 咳 白 粘 痰 , 无 恶 寒 发 热 , 无 胸 痛 , 无 恶 心 呕 吐 , 无 腹 痛 泻 , 纳 眠 可 , 二 便 正 常',
         'answer_start': 612}],
       'is_impossible': False},
      {'question': '诊 疗 经 过',
       'id': '0_0_1',
       'answers': [{'text': '入 院 后 积 极 完 善 相 关 检 查 , 排 除 手 术 禁 忌 后 , 于 2 0 1 8 - 1 0 - 1 9 在 全 麻 下 行 V A T S 左 上 肺 S 3 切 除 + 部 分 尖 段 切 除 术 , 手 术 顺 利 。 术 中 病 理 提 示 : ( 左 上 肺 前 段 + 尖 段 ) 取 材 一 块 示 原 位 腺 癌 伴 微 浸 润 , 待 肿 块 全 取 明 确 浸 润 灶 大 小 。 术 后 予 抗 感 染 、 止 咳',
         'answer_start': 710}],
       'is_impossible': False}],
     'context': '" 大 学 附 属 医 院 江 苏 区 亦 住 院 收 大 据 。 图 苏 3 2 0 6 0 1 南 通 大 学 附 属 医 院 出 院 记 录 姓 名 曹 国 林 住 院 号 1 5 9 6 5 5 4 病 区 胸 外 科 病 区 科 别 胸 外 科 床 号 2 2 床 年 龄 6 3 岁 入 院 日 期 2 0 1 8 年 1 0 月 1 7 日 出 院 日 期 2 0 1 8 年 1 0 月 2 5 日 手 术 名 称 : W A T S 左 上 肺 S 3 切 除 + 部 分 尖 段 切 除 术 手 术 日 期 : 2 0 1 8 年 1 0 月 

# transformers使用
https://mccormickml.com/2020/03/10/question-answering-with-a-fine-tuned-BERT/ 可视化

https://huggingface.co/transformers/task_summary.html

## train

In [ ]:
!python run_squad.py \
  --model_type bert \
  --model_name_or_path /media/data2/Bean/hosp_record/bert_30 \
  --do_train \
  --do_eval \
  --train_file med_examples.json \
  --predict_file med_examples.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 1 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir model_squad1

## test

In [ ]:
from transformers import pipeline

nlp = pipeline("question-answering", 'model_squad1')


context = """
fadfds
"""

result = nlp(question="现病史/入院情况", context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")